In [13]:
import requests
import pandas as pd

# Средневзвешенные процентные ставки по кредитам, предоставленным кредитными организациями физическим лицам
# Средневзвешенные процентные ставки по привлеченным кредитными организациями вкладам (депозитам) физических лиц
# Объем кредитов, предоставленных физическим лицам-резидентам
# Широкая денежная масса
# Номинальный курс на конец периода

def get_all_data(url, name):
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    raw_data = data.get("RawData", [])
    header_data = data.get("headerData", [])

    raw_data = pd.DataFrame(raw_data)
    header_data = pd.DataFrame(header_data)

    df_merged = header_data.merge(raw_data, left_on='id', right_on='colId', how='left')
    df_pivot = df_merged.pivot(index='dt', columns='elname', values='obs_val')
    df_pivot = df_pivot.reset_index().rename_axis(None, axis=1)
    month_dict = {'Январь':['01-01', '01.01'], 'Февраль':['02-01', '02.01'], 'Март':['03-01', '03.01'],
                  'Апрель':['04-01', '04.01'], 'Май':['05-01', '05.01'], 'Июнь':['06-01', '06.01'],
                  'Июль':['07-01', '07.01'], 'Август':['08-01', '08.01'], 'Сентябрь':['09-01', '09.01'],
                  'Октябрь':['10-01', '10.01'], 'Ноябрь':['11-01', '11.01'], 'Декабрь':['12-01', '12.01']
                  }
    try:
        df_pivot['dt'] = df_pivot['dt'].apply(lambda x: f'{x.split()[-1]}-{month_dict[x.split()[0]][0]}')
    except:
        df_pivot['dt'] = df_pivot['dt'].apply(lambda x: f"{x.split('.')[-1]}-{x.split('.')[1]}-{x.split('.')[0]}")
    df_pivot['dt'] = pd.to_datetime(df_pivot['dt'])
    df_pivot.columns = ['dt'] + [f"""{name} {col.lower()}""" for col in df_pivot.columns[1:]]
    return df_pivot.sort_values(by='dt')


url_1 = 'https://cbr.ru/dataservice/data?y1=2017&y2=2025&publicationId=14&datasetId=27&measureId=2'
url_2 = 'https://cbr.ru/dataservice/data?y1=2017&y2=2025&publicationId=18&datasetId=37&measureId=2'
url_3 = 'https://cbr.ru/dataservice/data?y1=2017&y2=2025&publicationId=20&datasetId=41&measureId=22'
url_4 = 'https://cbr.ru/dataservice/data?y1=2017&y2=2025&publicationId=5&datasetId=8&measureId='
url_5 = 'https://cbr.ru/dataservice/data?y1=2017&y2=2025&publicationId=33&datasetId=127&measureId='

d1 = get_all_data(url_1, 'Ставки по кредитам')
d2 = get_all_data(url_2, 'Ставки по вкладам')
d3 = get_all_data(url_3, 'Объем кредитов')
d4 = get_all_data(url_4, 'Широкая д.м.')
d5 = get_all_data(url_5, 'Курс')


In [14]:
total_df = d1.merge(d2, on='dt', how='outer').merge(d3, on='dt', how='outer').merge(d4, on='dt', how='outer').merge(d5, on='dt', how='outer')

In [15]:
total_df = total_df.rename(columns={"dt":"date"})

In [16]:
total_df.columns

Index(['date', 'Ставки по кредитам до 1 года, включая ''до востребования''',
       'Ставки по кредитам до 30 дней, включая ''до востребования''',
       'Ставки по кредитам от 1 года до 3 лет',
       'Ставки по кредитам от 181 дня до 1 года',
       'Ставки по кредитам от 31 до 90 дней',
       'Ставки по кредитам от 91 до 180 дней',
       'Ставки по кредитам свыше 1 года', 'Ставки по кредитам свыше 3 лет',
       'Ставки по вкладам "до востребования"',
       'Ставки по вкладам до 1 года, включая ''до востребования''',
       'Ставки по вкладам до 1 года, кроме ''до востребования''',
       'Ставки по вкладам до 30 дней, включая ''до востребования''',
       'Ставки по вкладам до 30 дней, кроме ''до востребования''',
       'Ставки по вкладам от 1 года до 3 лет',
       'Ставки по вкладам от 181 дня до 1 года',
       'Ставки по вкладам от 31 до 90 дней',
       'Ставки по вкладам от 91 до 180 дней', 'Ставки по вкладам свыше 1 года',
       'Ставки по вкладам свыше 3 лет', 'Объем к

In [121]:
total_df

,date,Ставки по кред до 1 год вкл до вос,Ставки по кред до 30 дне вкл до вос,Ставки по кред от 1 год до 3 лет,Ставки по кред от 181 дня до 1 год,Ставки по кред от 31 до 90 дне,Ставки по кред от 91 до 180 дне,Ставки по кред свы 1 год,Ставки по кред свы 3 лет,Ставки по вклад до вос,...,Объем кредит все,Широкая д.м. все,Широкая д.м. ден агр м2,Широкая д.м. деп и сбе сер,Широкая д.м. деп в ино вал дом хоз,Широкая д.м. деп в ино вал дру фин орг,Широкая д.м. деп в ино вал неф орг,Курс дол сша к руб на кон пер,Курс евр к руб на кон пер,Курс юан к руб на кон пер
0,2017-01-01,22.40,18.26,18.08,22.76,18.17,22.81,16.23,15.58,3.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.1600,64.4300,NaN
1,2017-02-01,21.06,18.22,17.87,21.18,17.70,23.33,16.00,15.45,3.42,...,NaN,51216.3,38017.1,485.2,5487.1,225.0,7001.8,57.9400,61.2600,NaN
2,2017-03-01,20.37,18.62,17.64,20.62,16.13,20.29,15.66,15.10,3.39,...,NaN,51124.8,38462.6,483.5,5292.8,214.9,6671.1,56.3800,60.6000,NaN
3,2017-04-01,20.57,17.81,17.54,20.87,16.42,20.53,15.42,14.86,2.60,...,NaN,50668.5,38555.5,488.7,5120.3,221.2,6282.8,56.9800,62.0400,NaN
4,2017-05-01,20.07,17.63,17.28,20.31,16.55,19.55,15.32,14.77,2.56,...,NaN,50860.2,38664.1,476.7,5257.6,256.3,6205.4,56.5200,62.9500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,2025-06-01,27.82,36.38,26.23,24.12,28.08,28.80,18.71,18.14,11.85,...,1815716.0,129859.9,118219.1,3.8,3090.7,270.9,8275.4,78.4685,92.2785,10.9433
102,2025-07-01,29.23,29.58,26.54,24.86,43.42,30.35,18.23,17.66,11.67,...,NaN,130662.8,119094.7,3.8,3101.0,257.0,8206.2,81.8347,94.9514,11.3683
103,2025-08-01,28.45,42.01,25.49,23.90,43.37,30.97,17.47,16.91,10.81,...,2085008.0,132246.3,120037.2,3.8,3242.2,260.2,8703.0,80.3316,94.0479,11.2713
104,2025-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,133832.0,121566.9,3.7,3197.1,243.1,8821.1,82.8676,97.1410,11.5978


In [109]:
total_df.to_sql('cbrf_data', con=connection(), if_exists='replace', index=False)

106

In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import create_engine, Table, MetaData, insert, text, TIMESTAMP
from sqlalchemy.dialects.postgresql import insert
import requests
from requests.exceptions import ConnectTimeout
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def connection():
    db_params = {
        'dbname': 'russian-stocks-prediction-ml-dl',
        'user': 'root',
        'password': 'groot',
        'host': '185.70.105.233',
        'port': '5432'
    }
    conn_str = f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}'
    return create_engine(conn_str)


def get_data(url, name):
    response = requests.get(url)
    response.raise_for_status()

    data = response.json()
    raw_data = data.get("RawData", [])
    header_data = data.get("headerData", [])

    raw_data = pd.DataFrame(raw_data)
    header_data = pd.DataFrame(header_data)

    df_merged = header_data.merge(raw_data, left_on='id', right_on='colId', how='left')
    df_pivot = df_merged.pivot(index='dt', columns='elname', values='obs_val')
    df_pivot = df_pivot.reset_index().rename_axis(None, axis=1)
    month_dict = {'Январь':['01-01', '01.01'], 'Февраль':['02-01', '02.01'], 'Март':['03-01', '03.01'],
                  'Апрель':['04-01', '04.01'], 'Май':['05-01', '05.01'], 'Июнь':['06-01', '06.01'],
                  'Июль':['07-01', '07.01'], 'Август':['08-01', '08.01'], 'Сентябрь':['09-01', '09.01'],
                  'Октябрь':['10-01', '10.01'], 'Ноябрь':['11-01', '11.01'], 'Декабрь':['12-01', '12.01']
                  }
    try:
        df_pivot['dt'] = df_pivot['dt'].apply(lambda x: f'{x.split()[-1]}-{month_dict[x.split()[0]][0]}')
    except:
        df_pivot['dt'] = df_pivot['dt'].apply(lambda x: f"{x.split('.')[-1]}-{x.split('.')[1]}-{x.split('.')[0]}")
    df_pivot['dt'] = pd.to_datetime(df_pivot['dt'])
    df_pivot.columns = ['dt'] + [f"""{name} {' '.join([word.strip("''").strip('""')[:3] for word in col.lower().split()])}""" for col in df_pivot.columns[1:]]
    return df_pivot.sort_values(by='dt')


def fetch_last_cbrf_data(cur_year):

    total_df = pd.DataFrame()

    url_1 = f'https://cbr.ru/dataservice/data?y1={cur_year}&y2={cur_year}&publicationId=14&datasetId=27&measureId=2'
    url_2 = f'https://cbr.ru/dataservice/data?y1={cur_year}&y2={cur_year}&publicationId=18&datasetId=37&measureId=2'
    url_3 = f'https://cbr.ru/dataservice/data?y1={cur_year}&y2={cur_year}&publicationId=20&datasetId=41&measureId=22'
    url_4 = f'https://cbr.ru/dataservice/data?y1={cur_year}&y2={cur_year}&publicationId=5&datasetId=8&measureId='
    url_5 = f'https://cbr.ru/dataservice/data?y1={cur_year}&y2={cur_year}&publicationId=33&datasetId=127&measureId='

    url_list = [url_1, url_2, url_3, url_4, url_5]
    names_list = ['Ставки по кред',
                  'Ставки по вклад',
                  'Объем кредит',
                  'Широкая д.м.',
                  'Курс'
                  ]

    for ind in range(len(names_list)):
        d = get_data(url_list[ind], names_list[ind])
        if ind == 0:
            total_df = d
        else:
            total_df = total_df.merge(d, on='dt', how='outer')
    total_df = total_df.rename(columns={'dt':'date'})
    return total_df


def update_db(df):
    orm_columns = cbrf_data.columns.keys()
    df.columns = orm_columns

    records = df.to_dict(orient='records')
    stmt = insert(cbrf_data).values(records)

    set_dict = {col: stmt.excluded[col] for col in df.columns if col != 'date'}
    stmt = stmt.on_conflict_do_update(
        index_elements=['date'],
        set_=set_dict
    )

    with connection().begin() as conn:
        conn.execute(stmt)


def main():
    while True:
        cur_year = datetime.now().year
        full_df = fetch_last_cbrf_data(cur_year)
        if len(full_df) > 0:
            update_db(full_df)
            print("Данные собраны и обновлены в БД.")
        else:
            print("Данные за текущий год пока отсутствуют.")
        print("Следующая итерация будет запущена через 14 дней.")
        time.sleep(14 * 24 * 3600)

metadata = MetaData()
cbrf_data = Table('cbrf_data', metadata, autoload_with=connection())

if __name__ == "__main__":
    main()


In [2]:
df = pd.read_sql("SELECT * FROM cbrf_data", connection())

In [4]:
df.sort_values(by='date')

,date,Cтавки по кред до 1 год вкл до вос,Cтавки по кред до 30 дне вкл до вос,Cтавки по кред от 1 год до 3 лет,Cтавки по кред от 181 дня до 1 год,Cтавки по кред от 31 до 90 дне,Cтавки по кред от 91 до 180 дне,Cтавки по кред свы 1 год,Cтавки по кред свы 3 лет,Cтавки по вклад до вос,...,Объем кредит все,Широкая д.м. все,Широкая д.м. ден агр м2,Широкая д.м. деп и сбе сер,Широкая д.м. деп в ино вал дом хоз,Широкая д.м. деп в ино вал дру фин о,Широкая д.м. деп в ино вал неф орг,Курс дол сша к руб на кон пер,Курс евр к руб на кон пер,Курс юан к руб на кон пер
0,2017-01-01,22.40,18.26,18.08,22.76,18.17,22.81,16.23,15.58,3.11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.1600,64.4300,NaN
1,2017-02-01,21.06,18.22,17.87,21.18,17.70,23.33,16.00,15.45,3.42,...,NaN,51216.3,38017.1,485.2,5487.1,225.0,7001.8,57.9400,61.2600,NaN
2,2017-03-01,20.37,18.62,17.64,20.62,16.13,20.29,15.66,15.10,3.39,...,NaN,51124.8,38462.6,483.5,5292.8,214.9,6671.1,56.3800,60.6000,NaN
3,2017-04-01,20.57,17.81,17.54,20.87,16.42,20.53,15.42,14.86,2.60,...,NaN,50668.5,38555.5,488.7,5120.3,221.2,6282.8,56.9800,62.0400,NaN
4,2017-05-01,20.07,17.63,17.28,20.31,16.55,19.55,15.32,14.77,2.56,...,NaN,50860.2,38664.1,476.7,5257.6,256.3,6205.4,56.5200,62.9500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2025-06-01,27.82,36.38,26.23,24.12,28.08,28.80,18.71,18.14,11.85,...,1815716.0,129859.9,118219.1,3.8,3090.7,270.9,8275.4,78.4685,92.2785,10.9433
96,2025-07-01,29.23,29.58,26.54,24.86,43.42,30.35,18.23,17.66,11.67,...,NaN,130662.8,119094.7,3.8,3101.0,257.0,8206.2,81.8347,94.9514,11.3683
97,2025-08-01,28.45,42.01,25.49,23.90,43.37,30.97,17.47,16.91,10.81,...,2085008.0,132246.3,120037.2,3.8,3242.2,260.2,8703.0,80.3316,94.0479,11.2713
98,2025-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,133832.0,121566.9,3.7,3197.1,243.1,8821.1,82.8676,97.1410,11.5978
